In [2]:
!nvidia-smi

Sun May 31 10:04:47 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import os
os.chdir('/content/drive/My Drive/GP/src')
print(os.getcwd())
print(os.listdir())

/content/drive/My Drive/GP/src
['nn.py', 'preprocessing.py', 'DRIVE_preprocess.py', 'DRIVE_data_loader.py', 'U_Net.py', 'U_Net1.py', '__pycache__', 'patch_extracting.py', 'U_Net2.py', 'unet2_10.pkl', '.ipynb_checkpoints', 'unet2_20.pkl', 'unet2_30.pkl', 'unet2_40.pkl', 'unet2_50.pkl', 'unet2_60.pkl', 'unet2_70.pkl', 'unet2_80.pkl', 'unet2_90.pkl', 'unet2_100.pkl', 'DRIVE_patch.py', 'unet2_110.pkl', 'unet2_120.pkl', 'Untitled0.ipynb', 'unet2_130.pkl', 'unet2_140.pkl', 'unet2_150.pkl', 'unet2_160.pkl', 'unet2_170.pkl', 'DRIVE_data_set.py', 'unet2_180.pkl', 'unet2_181.pkl', 'unet2_182.pkl', 'unet2_183.pkl', 'unet2_184.pkl', 'unet2_185.pkl', 'unet2_186.pkl', 'DRIVE_metrics.py', 'DRIVE_predict.py', 'segmentation.py', 'metrics.py', 'roc.jpg', 'DRIVE_train.py', 'unet2_187.pkl', 'unet2_188.pkl', 'unet2_189.pkl', 'unet2_190.pkl', 'unet2_191.pkl', 'unet2_192.pkl', 'unet2_193.pkl', 'unet2_194.pkl', 'model_mobile.py', 'unet2_195.pkl', 'DRIVE_unet2_train.ipynb', 'unet2_196.pkl']


In [0]:
!python DRIVE_train.py

tcmalloc: large alloc 3407872000 bytes == 0x50aa000 @  0x7fb1f709d1e7 0x7fb1f48faf71 0x7fb1f4963936 0x7fb1f496460e 0x7fb1f49fb98f 0x50ac25 0x50d390 0x508245 0x50a080 0x50aa7d 0x50d390 0x508245 0x50a080 0x50aa7d 0x50c5b9 0x508245 0x509642 0x595311 0x54a6ff 0x551b81 0x5aa6ec 0x50abb3 0x50d390 0x508245 0x50b403 0x635222 0x6352d7 0x638a8f 0x639631 0x4b0f40 0x7fb1f6c9ab97
tcmalloc: large alloc 3407872000 bytes == 0xd02aa000 @  0x7fb1f709d1e7 0x7fb1f48faf71 0x7fb1f4963936 0x7fb1f496460e 0x7fb1f49fb98f 0x50ac25 0x50d390 0x508245 0x50a080 0x50aa7d 0x50d390 0x508245 0x50a080 0x50aa7d 0x50c5b9 0x508245 0x509642 0x595311 0x54a6ff 0x551b81 0x5aa6ec 0x50abb3 0x50d390 0x508245 0x50b403 0x635222 0x6352d7 0x638a8f 0x639631 0x4b0f40 0x7fb1f6c9ab97
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/numpy/lib/npyio.py", line 453, in load
    pickle_kwargs=pickle_kwargs)
  File "/usr/local/lib/python3.6/dist-packages/numpy/lib/format.py", line 768, in read_array
    array.sh

In [0]:
# -*- coding: utf-8 -*-
"""
Created on Tue Mar 10 17:36:12 2020

@author: zll
"""


import os
from patch_extracting import get_imarr, extract
import numpy as np

mode = 'training'

img_path = '../data/DRIVE/' + mode + '/proc_imgs/'
truth_path = '../data/DRIVE/' + mode + '/1st_manual/'
write_path = '../data/DRIVE/' + mode + '/'


# get file name from image path
img_files = os.listdir(img_path)
# get file name from truth path
truth_files = os.listdir(truth_path)

# sort the file name list
img_files.sort()
truth_files.sort()

# combine image path with file names
img_files = [img_path + f for f in img_files]
truth_files = [truth_path + f for f in truth_files]


# get array
img_array = get_imarr(img_files)
truth_array = get_imarr(truth_files)

img_patchs = extract(img_array, (64, 64), 10, False)  # roughly 10k patchs per image
truth_patchs = extract(truth_array, (64, 64), 10, True)

np.save(write_path+'imgs', img_patchs)
np.save(write_path+'truth', truth_patchs)


In [0]:
# -*- coding: utf-8 -*-
"""
Created on Sat Mar 14 23:12:44 2020

@author: zll
"""
from DRIVE_data_set import DRIVE_data_set
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from U_Net2 import U_Net
from time import time

#################################
# training epoches
EPOCHES = 100
# epoches pre-trained
st_ep = 196
# if there is a pre_trained net
PRE_TRAIN = True
# pretrained net path
pretrained_net = 'unet2_195.pkl'
# batch size
batch_size=512
#################################

train_set = DRIVE_data_set(train=True)
loader = DataLoader(train_set, batch_size=batch_size)  # data loader


# use GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('device', device)

# creat and init unet
unet = U_Net()
if PRE_TRAIN:
    unet.load_state_dict(torch.load(pretrained_net))

unet.to(device)  # copy modle to GPU

# loss function
criterion = nn.CrossEntropyLoss().cuda()
# optimizer
optimizer = optim.SGD(unet.parameters(), lr=0.0001, momentum=0.9)

# training
for epoch in range(st_ep, st_ep + EPOCHES):
    start_time = time()
    running_loss = 0.0
    for i, data in enumerate(loader, 0):
        # get input
        inputs, lables = data
        lables = torch.tensor(lables, dtype=torch.long)
        lables = lables.squeeze(1)
        # copy data to GPU
        inputs, lables = inputs.to(device), lables.to(device)
        # set gradiant 0
        optimizer.zero_grad()

        # forwarding
        outputs = unet(inputs)
        # compute loss
        loss = criterion(outputs, lables)
        # backwarding
        loss.backward()
        # optimizing
        optimizer.step()

        # print states info
        running_loss += loss.item() * batch_size
    # print epoch loss and time
    print('[%d, loss: %.6f]' % (epoch, running_loss / train_set.len))
    print((time() - start_time) // 60, 'minutes per epoch.')
    # save model every epoch
    torch.save(unet.state_dict(), 'unet2_'+str(epoch)+'.pkl')


device cuda:0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


[196, loss: 0.103113]
7.0 minutes per epoch.
[197, loss: 0.102889]
7.0 minutes per epoch.
[198, loss: 0.102780]
7.0 minutes per epoch.
[199, loss: 0.102691]
7.0 minutes per epoch.
[200, loss: 0.102610]
7.0 minutes per epoch.
[201, loss: 0.102534]
7.0 minutes per epoch.
[202, loss: 0.102463]
7.0 minutes per epoch.
[203, loss: 0.102394]
7.0 minutes per epoch.
[204, loss: 0.102329]
7.0 minutes per epoch.
[205, loss: 0.102266]
7.0 minutes per epoch.
[206, loss: 0.102206]
7.0 minutes per epoch.
[207, loss: 0.102147]
7.0 minutes per epoch.
[208, loss: 0.102091]
7.0 minutes per epoch.
[209, loss: 0.102036]
7.0 minutes per epoch.
[210, loss: 0.101983]
7.0 minutes per epoch.
[211, loss: 0.101931]
7.0 minutes per epoch.
[212, loss: 0.101880]
7.0 minutes per epoch.
[213, loss: 0.101831]
7.0 minutes per epoch.
[214, loss: 0.101783]
7.0 minutes per epoch.
[215, loss: 0.101736]
7.0 minutes per epoch.
[216, loss: 0.101690]
7.0 minutes per epoch.
[217, loss: 0.101645]
7.0 minutes per epoch.
[218, loss